In [ ]:
!pip install openai --quiet
!pip install pydantic==1.10.11 --upgrade

In [ ]:
from sklearn.preprocessing import LabelEncoder
import openai
from openai import OpenAI
from tqdm import tqdm
from functools import partial
import pandas as pd
import time

le = LabelEncoder()
tqdm.pandas()

In [ ]:
uetcfs_data = pd.read_csv('/kaggle/input/uet-data/uetcfs_new.csv')

In [ ]:
uetcfs_data.loc[uetcfs_data['Sentiment'].isna(), 'Sentiment'] = '1'
uetcfs_data['sentiment'] = le.fit_transform(uetcfs_data['Sentiment'])
uetcfs_data = uetcfs_data[['content', 'sentiment', 'new_sentiment']]
uetcfs_data

In [ ]:
def generate_label(client, prompt):
    try:
        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {
              "role": "system",
                  "content": "You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.\
                              The output will be only one word, either 'positive', 'neutral' or 'negative'."
            },
            {
              "role": "user",
              "content": prompt
            }
          ],
          temperature=0,
          max_tokens=1,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print('Error:', e)
        return None

In [ ]:
uetcfs_data['new_sentiment'] = uetcfs_data['new_sentiment'].fillna('1')

In [ ]:
map_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

In [ ]:
stop_index = -1
count = 0
api_index = 0

for index, row in tqdm(uetcfs_data.iterrows(), total=uetcfs_data.shape[0]):
     if row['new_sentiment'] == None or row['new_sentiment'] == 'None' or row['new_sentiment'] == '1':
        if row['new_sentiment'] not in map_dict or row['new_sentiment'] not in map_dict.values():
            try:
                client = OpenAI(
                    api_key= # Add your OpenAI API key here,
                )
                prompt = row['content']
                new_label = generate_label(client, prompt)
                uetcfs_data.loc[index, 'new_sentiment'] = new_label
                count += 1
                print(api_index)
                if count == 3:
                    api_index = (api_index + 1) % (len(api_keys))
                    count = 0
                time.sleep(21)

            except:
                print("Early die")
                stop_index = index
                break

In [ ]:
count_none = 0
for index, row in tqdm(uetcfs_data.iterrows(), total=uetcfs_data.shape[0]):
    if row['new_sentiment'] == None or row['new_sentiment'] == 'None' or row['new_sentiment'] == '1' or row['new_sentiment'] not in map_dict:
            if index >= 4700:
                break
            else:
                count_none += 1
                
count_none           

In [ ]:
def tokenize_label(text):
    return map_dict[text]

In [ ]:
uetcfs_data['new_sentiment'] = uetcfs_data.new_sentiment.progress_apply(partial(tokenize_label),)

In [ ]:
uetcfs_data.to_csv('/kaggle/working/uetcfs_new.csv')